In [125]:
#import words.json
import json
import numpy as np
from string import ascii_lowercase as alphabet


def split(word):
    return [char for char in word]

knownLetters=split("gho")#all yellow or green
usedLetters=np.array(split("ghost")) #all used letters
usedLettersWithUnknownPosition= [[],[],[],[],[]] #only yellow used letters in correct position
lettersWithKnownPosition=["g","h","o","",""]#green

In [126]:
print(knownLetters)
print(usedLetters)
print(usedLettersWithUnknownPosition)
print(lettersWithKnownPosition)


['g', 'h', 'o']
['g' 'h' 'o' 's' 't']
[[], [], [], [], []]
['g', 'h', 'o', '', '']


In [127]:

with open('Words.json') as f:
    allWordleWords = json.load(f)


def filterWordsContainingAllKnownLetters(allWordleWords,knownLetters):
    filteredWordleWords=[]
    for word in allWordleWords:
        if all(letter in word for letter in knownLetters):
            filteredWordleWords.append(word)
    return filteredWordleWords

def filterWordsAccordingToKnownPositions(remainingWordleWords,lettersWithKnownPosition):
    if lettersWithKnownPosition == ["","","","",""]:
        return remainingWordleWords
    filteredWordleWords=[]
    #append all words that contain all letters in correct position
    for word in remainingWordleWords:
        allLettersInWordFit=[False]*5
        for letterIndex, letter in enumerate(word):
            if lettersWithKnownPosition[letterIndex] == "":
                allLettersInWordFit[letterIndex]=True
            elif letter == lettersWithKnownPosition[letterIndex]:
                allLettersInWordFit[letterIndex]=True
        if all(allLettersInWordFit):
            filteredWordleWords.append(word)
    return filteredWordleWords


    # for word in remainingWordleWords:
    #     allConditionsMet=False
    #     for index, letter in enumerate(word):
    #         if lettersWithKnownPosition[index]=="":
    #             pass
    #         elif letter in lettersWithKnownPosition[index]:
    #             allConditionsMet=True
    #         else:
    #             allConditionsMet=False
    #     if allConditionsMet:
    #         filteredWordleWords.append(word)
    return filteredWordleWords

#determine unique letters that cannot be in word
def getAvailableLetters(knownLetters,usedLetters):
    availableLetters=[]
    for letter in alphabet:
        if letter in knownLetters or letter not in usedLetters:
            availableLetters.append(letter)
    return availableLetters

def filterWordsAccordingToAvailableLetters(remainingWordleWords,availableLetters):
    filteredWordleWords=[]
    for word in remainingWordleWords:
        if all(letter in availableLetters for letter in word):
            filteredWordleWords.append(word)
    return filteredWordleWords

def availableLettersAccordingToPosition(usedLettersWithUnknownPosition):
    availableLetters=[[],[],[],[],[]]
    for letter in alphabet:
        for index, usedLetterArray in enumerate(usedLettersWithUnknownPosition):
            if letter not in usedLetterArray:
                availableLetters[index].append(letter)
    return availableLetters

def filterWordsAccordingToKnownLetterPositions(remainingWordleWords,availableLettersWithPosition):
    filteredWordleWords=[]
    
    for word in remainingWordleWords:
        allLettersAvailable=True
        for index, letter in enumerate(word):
            if letter not in availableLettersWithPosition[index]:
                allLettersAvailable=False
        if allLettersAvailable:
            filteredWordleWords.append(word)
    return filteredWordleWords

def split(word):
    return [char for char in word]


In [128]:
availableLettersWithPosition = availableLettersAccordingToPosition(usedLettersWithUnknownPosition)
availableLetters = getAvailableLetters(knownLetters,usedLetters)
remainingWordleWords = filterWordsContainingAllKnownLetters(allWordleWords,knownLetters)
remainingWordleWords = filterWordsAccordingToKnownPositions(remainingWordleWords,lettersWithKnownPosition)
remainingWordleWords = filterWordsAccordingToAvailableLetters(remainingWordleWords,availableLetters)
remainingWordleWords = filterWordsAccordingToKnownLetterPositions(remainingWordleWords,availableLettersWithPosition)

In [142]:
letterCountDict = {}
for letter in alphabet:
    letterCountDict[letter] = [0,0,0,0,0]


{'a': [0, 0, 0, 0, 0], 'b': [0, 0, 0, 0, 0], 'c': [0, 0, 0, 0, 0], 'd': [0, 0, 0, 0, 0], 'e': [0, 0, 0, 0, 0], 'f': [0, 0, 0, 0, 0], 'g': [0, 0, 0, 0, 0], 'h': [0, 0, 0, 0, 0], 'i': [0, 0, 0, 0, 0], 'j': [0, 0, 0, 0, 0], 'k': [0, 0, 0, 0, 0], 'l': [0, 0, 0, 0, 0], 'm': [0, 0, 0, 0, 0], 'n': [0, 0, 0, 0, 0], 'o': [0, 0, 0, 0, 0], 'p': [0, 0, 0, 0, 0], 'q': [0, 0, 0, 0, 0], 'r': [0, 0, 0, 0, 0], 's': [0, 0, 0, 0, 0], 't': [0, 0, 0, 0, 0], 'u': [0, 0, 0, 0, 0], 'v': [0, 0, 0, 0, 0], 'w': [0, 0, 0, 0, 0], 'x': [0, 0, 0, 0, 0], 'y': [0, 0, 0, 0, 0], 'z': [0, 0, 0, 0, 0]}


In [144]:
for word in allWordleWords:
    for index, letter in enumerate(word):
        letterCountDict[letter][index] += 1
print(letterCountDict)

{'a': [737, 2263, 1236, 1076, 680], 'b': [909, 81, 335, 243, 59], 'c': [922, 176, 392, 411, 127], 'd': [685, 84, 390, 471, 823], 'e': [303, 1628, 882, 2327, 1522], 'f': [598, 24, 178, 233, 82], 'g': [639, 76, 364, 423, 143], 'h': [489, 547, 120, 235, 370], 'i': [165, 1383, 1051, 880, 280], 'j': [202, 11, 46, 29, 3], 'k': [377, 95, 272, 503, 259], 'l': [577, 699, 848, 771, 477], 'm': [693, 188, 511, 402, 182], 'n': [325, 345, 964, 788, 532], 'o': [262, 2097, 994, 698, 389], 'p': [859, 231, 364, 418, 147], 'q': [79, 15, 13, 2, 4], 'r': [628, 940, 1200, 719, 673], 's': [1565, 93, 533, 516, 3958], 't': [815, 239, 616, 898, 727], 'u': [189, 1188, 667, 402, 67], 'v': [242, 52, 240, 156, 4], 'w': [413, 163, 271, 128, 64], 'x': [16, 57, 133, 12, 70], 'y': [181, 271, 213, 108, 1301], 'z': [105, 29, 142, 126, 32]}


In [131]:
normalizedLetterCountDict = {}
for letter in alphabet:
    normalizedLetterCountDict[letter] = [0,0,0,0,0]
    for index, count in enumerate(letterCountDict[letter]):
        normalizedLetterCountDict[letter][index] = count/len(remainingWordleWords)


In [132]:
#determine probability duplicate letters in remaining word list index is letter number of occurrences
letterMultiplesArray = {}
for letter in alphabet:
    letterMultiplesArray[letter] = [0,0,0,0,0]

In [133]:
for word in remainingWordleWords:
    for letter in word:
        letterMultiplesArray[letter][word.count(letter)-1] += 1

In [134]:
# normalise letterMultiplesArray
normalizedLetterMultiplesDict = {}
for letter in alphabet:
    normalizedLetterMultiplesDict[letter] = [0,0,0,0,0]
    for index, count in enumerate(letterMultiplesArray[letter]):
        normalizedLetterMultiplesDict[letter][index] = count/len(remainingWordleWords)

In [135]:
#rate all words in remainingWordleWords return dictionary with word as key and score as value
wordScoreDict = {}
for word in remainingWordleWords:
    score = 0
    for index, letter in enumerate(word):
        score += normalizedLetterCountDict[letter][index]
    
    wordScoreDict[word] = score

In [136]:
#rate all words according to multiple of letter
wordScoreMultipleDict = {}
for word in remainingWordleWords:
    score = 0
    for index, letter in enumerate(word):
        score += normalizedLetterMultiplesDict[letter][index]
    
    wordScoreMultipleDict[word] = score

In [137]:
#sort wordScoreDict by value
sortedWordScoreDict = sorted(wordScoreDict.items(), key=lambda x: x[1], reverse=True)
print(sortedWordScoreDict)

[('ghoul', 5.0)]


In [138]:
sortedWordScoreMultipleDict = sorted(wordScoreMultipleDict.items(), key=lambda x: x[1], reverse=True)
print(sortedWordScoreMultipleDict)

[('ghoul', 1.0)]


In [139]:
totalScoreDict = {}
for word in remainingWordleWords:
    for index, letter in enumerate(word):
        #count letter in word
        multiple=word.count(letter)
        totalScoreDict[word] = normalizedLetterCountDict[letter][index] + normalizedLetterMultiplesDict[letter][multiple]

In [140]:
#sort totalScoreDict by value
sortedTotalScoreDict = sorted(totalScoreDict.items(), key=lambda x: x[1], reverse=True)
print(sortedTotalScoreDict)

[('ghoul', 1.0)]


In [141]:
# #remainingWordleWords = sortRemainingWordleWordsAccordingToLetterFrequencyAndPosition(remainingWordleWords)
# def calculateLetterScoreDict(remainingWordleWords):
#     letters={}
#     for word in remainingWordleWords:
#         letterLevel = getLetterLevel(word)
#         for index, letter in enumerate(word):
#             #if letter in letters and letter not in duplicateLetters:
#             if letter in letters:
#                 letters[letter] = letterFrequencyArray(letter,letterLevel)
#             else:
#                 letters[letter] = 1
#     # for letter in letters:
#     #     letters[letter] = letters[letter]/len(remainingWordleWords)
#     return letters

# def letterFrequencyArray(letter,letterLevel):
#     "returns the frequency of letter in word in an array of five elements"
#     letterCount = letterLevel[letter]
#     for position in range(letterCount):
#         letterFrequencyArray[position] += 1
#     return letterFrequencyArray

# def getLetterLevel(word):
#     "returns a dictionary with count of letters in word as values and letter as key"
#     countedRepeatedLetters={}
#     for letter in word:
#         if letter in countedRepeatedLetters:
#             countedRepeatedLetters[letter] += 1
#         else:
#             countedRepeatedLetters[letter] = 1
#     return countedRepeatedLetters



# #give score to words according to letter scores
# def scoreWords(remainingWordleWords,letterScores):
#     wordScores={}
#     for word in remainingWordleWords:
#         wordScores[word] = sum(letterScores[letter] for letter in word)
#     return wordScores

# letterScores = calculateLetterScoreDict(remainingWordleWords)